In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import pymc3 as pm
import arviz as az

In [6]:
COPEPOD80_BIOMASS = pd.read_csv('/net/meso/work/svivier/Git/Data/Biomass_IM_final.csv')

In [7]:
COPEPOD80_df_clean = pd.read_csv('/net/meso/work/svivier/Git/Data/Abundance_IM_final.csv')

In [8]:
COPEPOD80_BIOMASS

,Carbon Mass
0,2.878586
1,17.761304
2,2.517407
3,11.215845
4,8.311358
...,...
1022,0.103977
1023,1.126274
1024,4.410456
1025,6.730864


In [9]:
COPEPOD80_df_clean

,Acartia,Acartia bifilosa,Acartia longiremis,Acartia spp.,Aetideopsis multiserrata,Aglantha digitale,Aglantha spp.,Alteutha interrupta,Augaptilus glacialis,Balanus balanus,...,Thysanoessa inermis,Thysanoessa longicaudata,Thysanoessa raschii,Thysanoessa spp.,Tisbe furcata,Undinella oblonga,Undinopsis spp.,Verruca spp.,Zaus goodsiri,Zaus spp.
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1023,0.0,0.0,0.0,0.19,0.0,0.0,0.0,0.0,0.0,0.0,...,0.37,0.0,0.0,0.0,0.0,0.0,0.19,0.38,0.0,0.0
1024,0.0,0.0,0.0,16.74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.93,0.0,0.0,0.0,0.0,0.0,2.79,3.72,0.0,0.0
1025,0.0,0.0,0.0,16.12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.62,31.63,0.0,0.0


# 1% + We create the simulated data: Simulated Abundance data = COPEPOD80_df_clean; Simulated Total Biomass data: COPEPOD80_df_clean x ICWs AtlantECO
We simulate data using our abundance data and the ICWs of AtlantECO=> We can test our model

Here, All species that has a maximum abundance <1% in each samples are grouped in Size groups (=1; <1; <<1)

In [12]:
groupes = {
    '=1': ['Aetideopsis multiserrata', 'Chiridius obtusifrons', 'Gaetanus brevispinus', 'Paraheterorhabdus compactus', 'Eukrohnia hamata', 'Rathkea spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    '<1': [ 'Mimocalanus distinctocephalus','Pseudochirella spectabilis', 'Aglantha spp.', 'Candacia armata', 'Haloptilus acutifrons', 'Heterorhabdus norvegicus', 'Mormonilla minor', 'Spinocalanus magnus', 'Spinocalanus spinosus'], 
    '<<1': ['Alteutha interrupta', 'Cyclopina longicornis', 'Danielssenia typica', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Laophonte perplexa', 'Lubbockia glacialis', 'Parathalestris croni', 'Parathalestris harpactoides', 'Rhynchothalestris helgolandica', 'Tegastes falcatus'],
    'inconnu': ['Euphysa spp.','Syllides spp.','Rathkea octopunctata','Discoconchoecia elegans','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.','Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.']


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [13]:
#=Priors corrigée: /10 les priors des copepods

priors = {
    '=1': {'mu': 1, 'sigma': 1},
    '<1': {'mu': 0.1, 'sigma': 0.1},
    '<<1': {'mu': 0.01, 'sigma': 0.01},
    'inconnu': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 5},
    'Evadne': {'sigma': 5},
    'Verruca': {'sigma':1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma':1},
    'Beroe': {'mu': 7.12, 'sigma': 10},
    'Acartia': {'mu': 0.003, 'sigma': 0.1},
    'Metridia': {'mu': 0.08, 'sigma': 0.1},
    'Paracalanus': {'mu': 0.0016, 'sigma': 0.01},
    'Temora': {'mu': 0.001, 'sigma': 0.01},
    'Oithona': {'mu': 0.0005, 'sigma': 0.01},
    'Spinocalanus': {'mu': 0.2, 'sigma': 0.2},
    'Centropages': {'mu': 0.0075, 'sigma': 0.1},
    'Sagitta': {'mu': 0.6, 'sigma': 1}, #Chaetognatha
    'Pseudocalanus': {'mu': 0.004, 'sigma': 0.1},
    'Thysanoessa': {'mu': 6, 'sigma': 6}, #Euphosiacea
    'Oikopleura': {'mu': 0.006, 'sigma': 0.01},
    'Microcalanus': {'mu': 0.0002, 'sigma': 0.01},
    'Microsetella spp.': {'mu': 0.0008, 'sigma': 0.01},
    'Calanus finmarchicus': {'mu': 0.2, 'sigma': 1},
    'Calanus glacialis': {'mu': 0.2, 'sigma': 1},
    'Calanus hyperboreus': {'mu': 1.7, 'sigma': 1},
    'Bradyidius': {'mu': 0.3, 'sigma': 1},
    'Semibalanus': {'sigma': 5},
    'Chaetognatha': {'mu': 1, 'sigma': 1},
    'Euchaeta':  {'mu': 0.14, 'sigma': 1},
    'Eukrohnia': {'mu': 1.4, 'sigma': 1.4},
    'Hyperiidea':{'mu': 20, 'sigma': 20},
    'Themisto': {'mu': 1.4, 'sigma': 1.4},
    'Bolinopsis': {'mu': 40, 'sigma': 10},
    'Monstrilla': {'mu': 0.1, 'sigma': 1},
    'Oncaea': {'mu': 0.001, 'sigma': 0.1},
    'Pagurus':{'sigma': 5},
    'Paraeuchaeta': {'mu': 1, 'sigma': 1},
    'Pleurobrachia': {'mu': 2, 'sigma': 2},
    'Podon': {'sigma': 1},
    'Polychaeta':{'mu': 0.05, 'sigma': 0.05},
    'Thalestris':{'mu': 0.001, 'sigma': 0.01},
    'Tisbe':{'mu': 0.001, 'sigma': 0.01},
    'Undinella':{'mu': 0.3, 'sigma': 1},
    'Augaptilus':{'mu': 0.3, 'sigma': 1}, 
    'Obelia':{'mu': 1, 'sigma': 1},
    'Plotocnide':{'mu': 1, 'sigma': 1}, 
    'Scaphocalanus':{'mu': 0.2, 'sigma': 1},
    'Meganyctiphanes':{'mu': 10, 'sigma': 10}, 
    'Harpacticus':{'mu': 0.001, 'sigma': 0.01},
    'Calanus spp.':{'mu': 0.7, 'sigma': 0.7}, 
    'Gammaridea':{'mu': 3, 'sigma': 3}, 
    'Hormiphora cucumis':{'mu': 10, 'sigma': 10},
    'Hyas spp.':{'sigma': 1}, 
    'Jaschnovia tolli':{'mu': 3, 'sigma': 3}, 
    'Microsetella spp.':{'mu': 0.001, 'sigma': 0.01},
    'Scolecithricella minor':{'mu': 0.1, 'sigma': 0.1}, 
    'Spionidae spp.':{'sigma': 1}, 
    'Temorites brevis':{'mu': 0.3, 'sigma': 1}
    
    
    


    
    

}

In [15]:
# SpeciesCmass: the ICWs used to simulate the total biomass of the samples = ICWs of AtlantECO if available, 1 if not.

import pandas as pd

# Créer une liste pour stocker les tuples (espèce, valeur)
data = []

for species, values in priors.items():
    # Sélectionner 'mu' s'il est présent, sinon 'sigma'
    value = values.get('mu', values['sigma'])
    data.append((species, value))

# Créer un DataFrame à partir de la liste des tuples
SpeciesCmass = pd.DataFrame(data, columns=['Species', 'Value']).set_index('Species')

# Affichage du DataFrame
SpeciesCmass


,Value
Species,
=1,1.0000
<1,0.1000
<<1,0.0100
inconnu,1.0000
Limacina,1.0000
Undinopsis,1.0000
Balanus,5.0000
Evadne,5.0000
Verruca,1.0000


In [18]:
import pandas as pd


# Assurez-vous que les indices de SpeciesCmass correspondent aux colonnes de COPEPOD80_reduite
SpeciesCmass = SpeciesCmass.reindex(COPEPOD80_reduite.columns)

# Calculer la biomasse totale pour chaque échantillon
RealTotalCmass = (COPEPOD80_reduite * SpeciesCmass['Value']).sum(axis=1)




In [19]:
RealTotalCmass

0        10.450000
1       105.770000
2         5.900000
3        51.610000
4        43.200000
           ...    
1022      0.004277
1023      7.553807
1024     35.969606
1025     75.938797
1026     14.852089
Length: 1027, dtype: float64

In [20]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hyas spp., Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Semibalanus, Bradyidius, Calanus hyperboreus, Calanus glacialis, Calanus finmarchicus, Microsetella spp., Microcalanus, Oikopleura, Thysanoessa, Pseudocalanus, Sagitta, Centropages, Spinocalanus, Oithona, Temora, Paracalanus, Metridia, Acartia, Beroe, Copepoda, Fritillaria, Verruca, Evadne, Balanus, Undinopsis, Limacina, inconnu, <<1, <1, =1]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 247 seconds.


In [21]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df



Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
=1,1.270,0.002,2.657
<1,0.130,0.000,0.263
<<1,0.013,0.000,0.026
inconnu,0.796,0.000,1.887
Limacina,0.921,0.369,1.415
Undinopsis,0.791,0.000,1.808
Balanus,4.765,3.867,5.622
Evadne,4.838,3.114,6.481
Verruca,0.918,0.375,1.458


In [22]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Aetideopsis multiserrata,1.270,0.002,2.657
Chiridius obtusifrons,1.270,0.002,2.657
Gaetanus brevispinus,1.270,0.002,2.657
Paraheterorhabdus compactus,1.270,0.002,2.657
Eukrohnia hamata,1.270,0.002,2.657
...,...,...,...
Hyas spp.,0.809,0.001,1.884
Jaschnovia tolli,3.760,0.002,7.870
Scolecithricella minor,0.131,0.000,0.266


In [24]:
groupes = {
    'unknow': ['Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (group 1%)', 'HDI 3% (group 1%)', 'HDI 97% (group 1%)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (group 1%)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (group 1%)': data_subset['HDI 3%'].mean(),
            'HDI 97% (group 1%)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (group 1%)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (group 1%)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (group 1%)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
f1=grouped_results.copy()


In [34]:
f1 = pd.concat([f1, SpeciesCmass], axis=1)
f1

,Biomasse moyenne (group 1%),HDI 3% (group 1%),HDI 97% (group 1%),Value,Value
unknow,0.7973,0.0001,1.8867,1.0000,1.0000
Acartia,0.0740,0.0000,0.1720,0.0030,0.0030
Metridia,0.0950,0.0020,0.1910,0.0800,0.0800
Microcalanus,0.0080,0.0000,0.0190,0.0002,0.0002
Temora,0.0080,0.0000,0.0190,0.0010,0.0010
...,...,...,...,...,...
Hormiphora cucumis,10.9720,0.5870,20.4330,10.0000,10.0000
Jaschnovia tolli,3.7600,0.0020,7.8700,3.0000,3.0000
Scolecithricella minor,0.1310,0.0000,0.2660,0.1000,0.1000
Spionidae spp.,0.7920,0.0010,1.9070,1.0000,1.0000


# non % + priors

In [79]:
groupes = {
    'unknow': ['Euphysa spp.','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [80]:
priors = {
    'unknow': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 5},
    'Evadne': {'sigma': 5},
    'Verruca': {'sigma':1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma':1},
    'Beroe': {'mu': 7.12, 'sigma': 10},
    'Acartia': {'mu': 0.003, 'sigma': 0.1},
    'Metridia': {'mu': 0.08, 'sigma': 0.1},
    'Paracalanus': {'mu': 0.0016, 'sigma': 0.1},
    'Temora': {'mu': 0.001, 'sigma': 0.01},
    'Oithona': {'mu': 0.0005, 'sigma': 0.01},
    'Spinocalanus': {'mu': 0.2, 'sigma': 1},
    'Centropages': {'mu': 0.0075, 'sigma': 0.01},
    'Sagitta': {'mu': 0.6, 'sigma': 1}, #Chaetognatha
    'Pseudocalanus': {'mu': 0.004, 'sigma': 0.1},
    'Thysanoessa': {'mu': 6, 'sigma': 6}, #Euphosiacea
    'Oikopleura': {'mu': 0.006, 'sigma': 0.01},
    'Microcalanus': {'mu': 0.0002, 'sigma': 0.01},
    'Microsetella spp.': {'mu': 0.0008, 'sigma': 0.01},
    'Calanus finmarchicus': {'mu': 0.2, 'sigma': 1},
    'Calanus glacialis': {'mu': 0.2, 'sigma': 1},
    'Calanus hyperboreus': {'mu': 1.7, 'sigma': 2},
    'Bradyidius': {'mu': 0.3, 'sigma': 1},
    'Semibalanus': {'sigma': 5},
    'Chaetognatha': {'mu': 1, 'sigma': 1},
    'Euchaeta':  {'mu': 0.14, 'sigma': 1},
    'Eukrohnia': {'mu': 1.4, 'sigma': 1.4},
    'Hyperiidea':{'mu': 20, 'sigma': 20},
    'Themisto': {'mu': 1.4, 'sigma': 1.4},
    'Bolinopsis': {'mu': 40, 'sigma': 10},
    'Monstrilla': {'mu': 0.1, 'sigma': 1},
    'Oncaea': {'mu': 0.001, 'sigma': 0.1},
    'Pagurus':{'sigma': 5},
    'Paraeuchaeta': {'mu': 1, 'sigma': 1},
    'Pleurobrachia': {'mu': 2, 'sigma': 2},
    'Podon': {'sigma': 1},
    'Polychaeta':{'mu': 0.05, 'sigma': 0.05},
    'Thalestris':{'mu': 0.001, 'sigma': 0.01},
    'Tisbe':{'mu': 0.001, 'sigma': 0.01},
    'Undinella':{'mu': 0.3, 'sigma': 1},
    'Augaptilus':{'mu': 0.3, 'sigma': 1}, 
    'Obelia':{'mu': 1, 'sigma': 1},
    'Plotocnide':{'mu': 1, 'sigma': 1}, 
    'Scaphocalanus':{'mu': 0.2, 'sigma': 1},
    'Meganyctiphanes':{'mu': 10, 'sigma': 10}, 
    'Harpacticus':{'mu': 0.001, 'sigma': 0.01},
    'Calanus spp.':{'mu': 0.7, 'sigma': 0.7}, 
    'Gammaridea':{'mu': 3, 'sigma': 3}, 
    'Hormiphora cucumis':{'mu': 10, 'sigma': 10},
    'Jaschnovia tolli':{'mu': 3, 'sigma': 3}, 
    'Scolecithricella minor':{'mu': 0.1, 'sigma': 0.1}, 
    'Spionidae spp.':{'sigma': 1}, 
    'Temorites brevis':{'mu': 0.3, 'sigma': 1},
    'Aetideopsis multiserrata':{'mu': 0.3, 'sigma': 1},
    'Chiridius obtusifrons': {'mu': 0.3, 'sigma': 1},
    'Gaetanus brevispinus': {'mu': 0.2, 'sigma': 1}, 
    'Paraheterorhabdus compactus': {'mu': 0.3, 'sigma': 1},
    'Aglantha spp.':{'mu': 1, 'sigma': 1},
    'Candacia armata':{'mu': 0.1, 'sigma': 0.1},
    'Haloptilus acutifrons': {'mu': 0.1, 'sigma': 0.1},
    'Heterorhabdus norvegicus': {'mu': 1, 'sigma': 1},
    'Mormonilla minor': {'mu': 0.1, 'sigma': 0.1},
    'Rathkea': {'mu': 1, 'sigma': 1}, 
    'Sarsia': {'mu': 1, 'sigma': 1}, 
    'Alteutha interrupta': {'mu': 0.001, 'sigma': 0.001},
    'Cyclopina longicornis': {'mu': 0.001, 'sigma': 0.001},
    'Danielssenia typica': {'mu': 0.001, 'sigma': 0.001},
    'Zaus': {'mu': 0.001, 'sigma': 0.001}, 
    'Laophonte perplexa':{'mu': 0.001, 'sigma': 0.001},
    'Lubbockia glacialis':{'mu': 0.001, 'sigma': 0.001},
    'Parathalestris': {'mu': 0.001, 'sigma': 0.001},
    'Rhynchothalestris helgolandica': {'mu': 0.001, 'sigma': 0.001},
    'Tegastes falcatus': {'mu': 0.001, 'sigma': 0.001},
    'Mimocalanus distinctocephalus': {'mu': 0.03, 'sigma': 0.1},
    'Pseudochirella spectabilis': {'mu': 0.03, 'sigma': 0.1},
    'Discoconchoecia elegans' : {'mu': 0.0015, 'sigma': 0.1},
    'Spinocalanus sp.' : {'mu': 0.2, 'sigma': 1}
    
    
    
    


    
    

}

In [81]:
import pandas as pd

# Créer une liste pour stocker les tuples (espèce, valeur)
data = []

for species, values in priors.items():
    # Sélectionner 'mu' s'il est présent, sinon 'sigma'
    value = values.get('mu', values['sigma'])
    data.append((species, value))

# Créer un DataFrame à partir de la liste des tuples
SpeciesCmass = pd.DataFrame(data, columns=['Species', 'Value']).set_index('Species')

# Affichage du DataFrame
SpeciesCmass


,Value
Species,
unknow,1.0000
Limacina,1.0000
Undinopsis,1.0000
Balanus,5.0000
Evadne,5.0000
...,...
Tegastes falcatus,0.0010
Mimocalanus distinctocephalus,0.0300
Pseudochirella spectabilis,0.0300


In [82]:
import pandas as pd


# Assurez-vous que les indices de SpeciesCmass correspondent aux colonnes de COPEPOD80_reduite
SpeciesCmass = SpeciesCmass.reindex(COPEPOD80_reduite.columns)

# Calculer la biomasse totale pour chaque échantillon
RealTotalCmass = (COPEPOD80_reduite * SpeciesCmass['Value']).sum(axis=1)


In [83]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Spinocalanus sp., Discoconchoecia elegans, Pseudochirella spectabilis, Mimocalanus distinctocephalus, Tegastes falcatus, Rhynchothalestris helgolandica, Parathalestris, Lubbockia glacialis, Laophonte perplexa, Zaus, Danielssenia typica, Cyclopina longicornis, Alteutha interrupta, Sarsia, Rathkea, Mormonilla minor, Heterorhabdus norvegicus, Haloptilus acutifrons, Candacia armata, Aglantha spp., Paraheterorhabdus compactus, Gaetanus brevispinus, Chiridius obtusifrons, Aetideopsis multiserrata, Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Se

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 415 seconds.


In [84]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df


Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,0.797,0.001,1.857
Limacina,0.910,0.333,1.439
Undinopsis,0.800,0.001,1.867
Balanus,4.770,3.874,5.653
Evadne,4.845,3.048,6.561
...,...,...,...
Tegastes falcatus,0.001,0.000,0.003
Mimocalanus distinctocephalus,0.091,0.000,0.199
Pseudochirella spectabilis,0.091,0.000,0.209


In [85]:
R1= results_df.copy()

# non% + Prior non informative

In [86]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [87]:
priors = {
    'unknow': {'sigma': 1},
    'Limacina': {'sigma': 1},
    'Undinopsis': {'sigma': 1},
    'Balanus': {'sigma': 1},
    'Evadne': {'sigma': 1},
    'Verruca': {'sigma': 1},
    'Fritillaria': {'sigma': 1},
    'Copepoda':{'sigma': 1},
    'Beroe': {'sigma': 1},
    'Acartia': {'sigma': 1},
    'Metridia': {'sigma': 1},
    'Paracalanus': {'sigma': 1},
    'Temora': {'sigma': 1},
    'Oithona': {'sigma': 1},
    'Spinocalanus': {'sigma': 1},
    'Centropages': {'sigma': 1},
    'Sagitta': {'sigma': 1},
    'Pseudocalanus': {'sigma': 1},
    'Thysanoessa': {'sigma': 1},
    'Oikopleura': {'sigma': 1},
    'Microcalanus': {'sigma': 1},
    'Microsetella spp.': {'sigma': 1},
    'Calanus finmarchicus': {'sigma': 1},
    'Calanus glacialis': {'sigma': 1},
    'Calanus hyperboreus': {'sigma': 1},
    'Bradyidius': {'sigma': 1},
    'Semibalanus': {'sigma': 1},
    'Chaetognatha': {'sigma': 1},
    'Euchaeta':  {'sigma': 1},
    'Eukrohnia': {'sigma': 1},
    'Hyperiidea':{'sigma': 1},
    'Themisto': {'sigma': 1},
    'Bolinopsis': {'sigma': 1},
    'Monstrilla': {'sigma': 1},
    'Oncaea': {'sigma': 1},
    'Pagurus':{'sigma': 1},
    'Paraeuchaeta': {'sigma': 1},
    'Pleurobrachia': {'sigma': 1},
    'Podon': {'sigma': 1},
    'Polychaeta':{'sigma': 1},
    'Thalestris':{'sigma': 1},
    'Tisbe':{'sigma': 1},
    'Undinella':{'sigma': 1},
    'Augaptilus':{'sigma': 1},
    'Obelia':{'sigma': 1},
    'Plotocnide':{'sigma': 1}, 
    'Scaphocalanus':{'sigma': 1},
    'Meganyctiphanes':{'sigma': 1},
    'Harpacticus':{'sigma': 1},
    'Calanus spp.':{'sigma': 1},
    'Gammaridea':{'sigma': 1},
    'Hormiphora cucumis':{'sigma': 1},
    'Jaschnovia tolli':{'sigma': 1},
    'Scolecithricella minor':{'sigma': 1},
    'Spionidae spp.':{'sigma': 1},
    'Temorites brevis':{'sigma': 1},
    'Aetideopsis multiserrata':{'sigma': 1},
    'Chiridius obtusifrons': {'sigma': 1},
    'Gaetanus brevispinus': {'sigma': 1}, 
    'Paraheterorhabdus compactus': {'sigma': 1},
    'Aglantha spp.':{'sigma': 1},
    'Candacia armata':{'sigma': 1},
    'Haloptilus acutifrons': {'sigma': 1},
    'Heterorhabdus norvegicus': {'sigma': 1},
    'Mormonilla minor': {'sigma': 1},
    'Rathkea': {'sigma': 1}, 
    'Sarsia':{'sigma': 1}, 
    'Alteutha interrupta': {'sigma': 1},
    'Cyclopina longicornis': {'sigma': 1},
    'Danielssenia typica': {'sigma': 1},
    'Zaus': {'sigma': 1}, 
    'Laophonte perplexa':{'sigma': 1},
    'Lubbockia glacialis':{'sigma': 1},
    'Parathalestris': {'sigma': 1},
    'Rhynchothalestris helgolandica': {'sigma': 1},
    'Tegastes falcatus': {'sigma': 1},
    'Mimocalanus distinctocephalus': {'sigma': 1},
    'Pseudochirella spectabilis': {'sigma': 1},
    'Discoconchoecia elegans' : {'sigma': 1},
    'Spinocalanus sp.' : {'sigma': 1}
    
    
    
    


    
    

}

In [ ]:
# Is all species in COPEPOD80_reduite has a prior?
especes_dans_COPEPOD80_reduite = set(COPEPOD80_reduite.columns)
especes_absentes_dans_priors = especes_dans_COPEPOD80_reduite - set(priors.keys())
especes_absentes_dans_priors

In [ ]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Spinocalanus sp., Discoconchoecia elegans, Pseudochirella spectabilis, Mimocalanus distinctocephalus, Tegastes falcatus, Rhynchothalestris helgolandica, Parathalestris, Lubbockia glacialis, Laophonte perplexa, Zaus, Danielssenia typica, Cyclopina longicornis, Alteutha interrupta, Sarsia, Rathkea, Mormonilla minor, Heterorhabdus norvegicus, Haloptilus acutifrons, Candacia armata, Aglantha spp., Paraheterorhabdus compactus, Gaetanus brevispinus, Chiridius obtusifrons, Aetideopsis multiserrata, Temorites brevis, Spionidae spp., Scolecithricella minor, Jaschnovia tolli, Hormiphora cucumis, Gammaridea, Calanus spp., Harpacticus, Meganyctiphanes, Scaphocalanus, Plotocnide, Obelia, Augaptilus, Undinella, Tisbe, Thalestris, Polychaeta, Podon, Pleurobrachia, Paraeuchaeta, Pagurus, Oncaea, Monstrilla, Bolinopsis, Themisto, Hyperiidea, Eukrohnia, Euchaeta, Chaetognatha, Se

In [45]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,0.814,0.000,1.911
Limacina,0.825,0.269,1.373
Undinopsis,1.141,0.024,2.386
Balanus,3.805,2.991,4.774
Evadne,2.568,1.268,3.941
...,...,...,...
Tegastes falcatus,0.794,0.000,1.906
Mimocalanus distinctocephalus,0.792,0.002,1.873
Pseudochirella spectabilis,0.795,0.000,1.821


In [46]:
R2=results_df.copy

# 6 groups

In [50]:
groupes = {
    
    'unknow': ['Euphysa spp.','Spionidae spp.','Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione', 'Balanus balanus', 'Balanus crenatus', 'Balanus spp.','Evadne nordmanni', 'Evadne spp.','Podon spp.', 'Podon leuckartii','Verruca spp.', 'Pagurus spp.', 'Semibalanus balanoides', 'Polychaeta','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Other':['Hormiphora cucumis', 'Gammaridea', 'Discoconchoecia elegans','Parasagitta elegans', 'Sagitta spp.', 'Eukrohnia spp.', 'Eukrohnia hamata', 'Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.', 'Hyperia spp.', 'Hyperiidea', 'Beroe spp.', 'Bolinopsis spp.', 'Pleurobrachia spp.', 'Chaetognatha', 'Obelia geniculata',  'Obelia longissima', 'Obelia spp.', 'Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.', 'Themisto abyssorum', 'Themisto spp.', 'Thysanoessa inermis', 'Thysanoessa spp.', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Meganyctiphanes norvegica', 'Meganyctiphanes spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa', 'Plotocnide borealis', 'Plotocnide spp.', 'Rathkea octopunctata', 'Rathkea spp.'],
    '0.0001': ['Temora longicornis', 'Oithona similis', 'Oithona atlantica','Microsetella spp.', 'Laophonte perplexa', 'Cyclopina longicornis', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Tisbe furcata', 'Thalestris gibba','Tegastes falcatus',  'Temora spp.', 'Rhynchothalestris helgolandica', 'Parathalestris croni', 'Parathalestris harpactoides',  'Oithona spp.', 'Oithona frigida', 'Harpacticus littoralis', 'Harpacticus spp.', 'Danielssenia typica', 'Alteutha interrupta'],
    '0.001': ['Pseudocalanus elongatus', 'Microcalanus pusillus', 'Microcalanus pygmaeus', 'Acartia spp.', 'Acartia longiremis', 'Lubbockia glacialis', 'Pseudocalanus gracilis', 'Pseudocalanus spp.', 'Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.',  'Microcalanus spp.', 'Centropages hamatus', 'Centropages spp.', 'Centropages typicus', 'Acartia bifilosa',  'Aglantha digitale', 'Acartia', 'Paracalanus spp.', 'Aglantha spp.'],
    '0.1': [ 'Metridia lucens', 'Metridia longa', 'Calanus glacialis', 'Calanus finmarchicus', 'Mormonilla minor', 'Mimocalanus distinctocephalus', 'Copepoda','Spinocalanus sp.', 'Monstrilla spp.', 'Undinopsis spp.', 'Euchaeta spp.', 'Undinella oblonga', 'Temorites brevis', 'Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis', 'Scolecithricella minor', 'Scaphocalanus brevicornis', 'Scaphocalanus magnus', 'Pseudochirella spectabilis', 'Paraheterorhabdus compactus', 'Jaschnovia tolli', 'Heterorhabdus norvegicus', 'Haloptilus acutifrons', 'Gaetanus brevispinus', 'Chiridius obtusifrons', 'Candacia armata',   'Bradyidius similis', 'Augaptilus glacialis', 'Aetideopsis multiserrata',  'Metridia spp.', 'Calanus spp.'],
    '1': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica', 'Calanus hyperboreus']
    
    
    


}
COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [51]:
priors = {
    'unknow': {'sigma': 1},
    'Other':{'sigma': 1},
    '0.0001':{'mu': 0.0001, 'sigma':0.1},
    '0.001':{'mu': 0.001, 'sigma': 0.1 },
    '0.1':{'mu': 0.1, 'sigma': 1},
    '1':{'mu': 1, 'sigma': 1}
}

In [52]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [1, 0.1, 0.001, 0.0001, Other, unknow]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [53]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,1.106,0.786,1.430
Other,4.931,4.774,5.103
0.0001,0.008,0.000,0.022
0.001,0.040,0.028,0.052
0.1,0.191,0.181,0.200
1,1.648,0.669,2.651


In [54]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Euphysa spp.,1.106,0.786,1.430
Spionidae spp.,1.106,0.786,1.430
Limacina retroversa,1.106,0.786,1.430
Limacina spp.,1.106,0.786,1.430
Limacina helicina,1.106,0.786,1.430
...,...,...,...
Metridia spp.,0.191,0.181,0.200
Calanus spp.,0.191,0.181,0.200
Paraeuchaeta glacialis,1.648,0.669,2.651


In [58]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (group 5%)', 'HDI 3% (group 5%)', 'HDI 97% (group 5%)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (group 5%)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (group 5%)': data_subset['HDI 3%'].mean(),
            'HDI 97% (group 5%)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (group 5%)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (group 5%)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (group 5%)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
grouped_results

,Biomasse moyenne (group 5%),HDI 3% (group 5%),HDI 97% (group 5%)
unknow,1.106,0.786,1.430
Acartia,0.040,0.028,0.052
Metridia,0.191,0.181,0.200
Microcalanus,0.040,0.028,0.052
Temora,0.008,0.000,0.022
...,...,...,...
Chiridius obtusifrons,0.191,0.181,0.200
Candacia armata,0.191,0.181,0.200
Aetideopsis multiserrata,0.191,0.181,0.200
Calanus spp.,0.191,0.181,0.200


In [59]:
R3=grouped_results.copy()

# 6 groups + dominant species

In [76]:
groupes = {
    
    'unknow': ['Euphysa spp.', 'Spionidae spp.','Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione', 'Balanus balanus', 'Balanus crenatus', 'Balanus spp.','Evadne nordmanni', 'Evadne spp.','Podon spp.', 'Podon leuckartii','Verruca spp.', 'Pagurus spp.', 'Semibalanus balanoides', 'Polychaeta','Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Other':['Hormiphora cucumis', 'Gammaridea', 'Discoconchoecia elegans','Parasagitta elegans', 'Sagitta spp.', 'Eukrohnia spp.', 'Eukrohnia hamata', 'Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.', 'Hyperia spp.', 'Hyperiidea', 'Beroe spp.', 'Bolinopsis spp.', 'Pleurobrachia spp.', 'Chaetognatha', 'Obelia geniculata',  'Obelia longissima', 'Obelia spp.', 'Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.', 'Themisto abyssorum', 'Themisto spp.', 'Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Meganyctiphanes norvegica', 'Meganyctiphanes spp.', 'Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa', 'Plotocnide borealis', 'Plotocnide spp.', 'Rathkea octopunctata', 'Rathkea spp.'],
    '0.0001': ['Microsetella spp.', 'Laophonte perplexa', 'Cyclopina longicornis', 'Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.', 'Tisbe furcata', 'Thalestris gibba','Tegastes falcatus',  'Temora spp.', 'Rhynchothalestris helgolandica', 'Parathalestris croni', 'Parathalestris harpactoides',  'Oithona spp.', 'Oithona frigida', 'Harpacticus littoralis', 'Harpacticus spp.', 'Danielssenia typica', 'Alteutha interrupta'],
    '0.001': ['Lubbockia glacialis', 'Pseudocalanus gracilis', 'Pseudocalanus spp.', 'Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.',  'Microcalanus spp.', 'Centropages hamatus', 'Centropages spp.', 'Centropages typicus', 'Acartia bifilosa',  'Aglantha digitale', 'Acartia', 'Paracalanus spp.', 'Aglantha spp.'],
    '0.1': ['Mormonilla minor', 'Mimocalanus distinctocephalus', 'Copepoda','Spinocalanus sp.', 'Monstrilla spp.', 'Undinopsis spp.', 'Euchaeta spp.', 'Undinella oblonga', 'Temorites brevis', 'Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis', 'Scolecithricella minor', 'Scaphocalanus brevicornis', 'Scaphocalanus magnus', 'Pseudochirella spectabilis', 'Paraheterorhabdus compactus', 'Jaschnovia tolli', 'Heterorhabdus norvegicus', 'Haloptilus acutifrons', 'Gaetanus brevispinus', 'Chiridius obtusifrons', 'Candacia armata',   'Bradyidius similis', 'Augaptilus glacialis', 'Aetideopsis multiserrata',  'Metridia spp.', 'Calanus spp.'],
    '1': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Calanus fimarchicus': ['Calanus finmarchicus'],
    'Calanus glacialis': ['Calanus glacialis'],
    'Calanus hyperboreus': [ 'Calanus hyperboreus'],
    'Oithona atlantica': ['Oithona atlantica'],
    'Oithona similis': ['Oithona similis'],
    'Metridia longa': ['Metridia longa'],
    'Metridia lucens': [ 'Metridia lucens'],
    'Acartia longiremis': ['Acartia longiremis'],
    'Acartia spp.': ['Acartia spp.'],
    'Microcalanus pusillus':['Microcalanus pusillus'],
    'Microcalanus pygmaeus':['Microcalanus pygmaeus'],
    'Pseudocalanus elongatus':['Pseudocalanus elongatus'],
    'Temora longicornis':['Temora longicornis'],
    'Thysanoessa spp.':['Thysanoessa spp.']
    
    
    


}

COPEPOD80_reduite = pd.DataFrame(index=COPEPOD80_df_clean.index)

for groupe, especes in groupes.items():
    # Vérifier si les espèces du groupe sont présentes dans le DataFrame d'abondance
    especes_presentes = [espece for espece in especes if espece in COPEPOD80_df_clean.columns]
    if especes_presentes:
        # Somme des abondances pour les espèces présentes dans le groupe
        COPEPOD80_reduite[groupe] = COPEPOD80_df_clean[especes_presentes].sum(axis=1)

# Ajouter les espèces qui ne sont pas dans les groupes
for espece in COPEPOD80_df_clean.columns:
    if not any(espece in especes for especes in groupes.values()):
        COPEPOD80_reduite[espece] = COPEPOD80_df_clean[espece]

In [77]:
priors = {
    'unknow': {'sigma': 1},
    'Other':{'sigma': 1},
    '0.0001':{'mu': 0.0001, 'sigma':0.1},
    '0.001':{'mu': 0.001, 'sigma': 0.1 },
    '0.1':{'mu': 0.1, 'sigma': 1},
    '1':{'mu': 1, 'sigma': 1},
    'Calanus fimarchicus': {'mu': 0.2, 'sigma':1 },
    'Calanus glacialis': {'mu': 0.2, 'sigma':1 },
    'Calanus hyperboreus': {'mu': 2, 'sigma':1 },
    'Oithona atlantica': {'mu': 0.0001, 'sigma':0.1 },
    'Oithona similis': {'mu': 0.0001, 'sigma':0.1},
    'Metridia longa': {'mu': 0.1, 'sigma':0.1},
    'Metridia lucens': {'mu': 0.1, 'sigma':0.1},
    'Acartia longiremis': {'mu': 0.001, 'sigma':0.1},
    'Acartia spp.': {'mu': 0.001, 'sigma':0.1},
    'Microcalanus pusillus':{'mu': 0.001, 'sigma':0.1},
    'Microcalanus pygmaeus':{'mu': 0.001, 'sigma':0.1},
    'Pseudocalanus elongatus':{'mu': 0.001, 'sigma':0.1},
    'Temora longicornis':{'mu': 0.0001, 'sigma':0.1},
    'Thysanoessa spp.':{'mu': 6, 'sigma':6},
}

In [78]:
ABONDANCE_sampled = COPEPOD80_reduite.values.astype(float)
BIOMASS_sampled = RealTotalCmass.values.flatten()

with pm.Model() as model:
    biomasses = {}
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    

    for species, params in priors.items():
        # Utiliser une distribution normale si un 'mu' est spécifié
        if 'mu' in params:
            biomasses[species] = PositiveNormal(species, mu=params['mu'], sigma=params['sigma'])
        else:
            # Utiliser une distribution HalfNormal sinon
            biomasses[species] = pm.HalfNormal(species, sigma=params['sigma'])

    # Créer un vecteur de biomasses dans le même ordre que les colonnes de ABONDANCE_sampled
    biomass_vector = [biomasses[species] for species in COPEPOD80_reduite.columns if species in biomasses]

# Calculer la biomasse prédite
    predicted_biomass = pm.math.dot(ABONDANCE_sampled, pm.math.stack(biomass_vector))


    # Likelihood: Comparaison de la biomasse prédite avec la biomasse observée
    observed = pm.Normal('observed', mu=predicted_biomass, sigma=50, observed=BIOMASS_sampled)

    # Exécution de l'échantillonneur
    trace = pm.sample(1000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Thysanoessa spp., Temora longicornis, Pseudocalanus elongatus, Microcalanus pygmaeus, Microcalanus pusillus, Acartia spp., Acartia longiremis, Metridia lucens, Metridia longa, Oithona similis, Oithona atlantica, Calanus hyperboreus, Calanus glacialis, Calanus fimarchicus, 1, 0.1, 0.001, 0.0001, Other, unknow]


KeyError: 0

In [63]:
summarry=pm.summary(trace)
# Extraire les moyennes et les intervalles HDI pour chaque variable de biomasse
biomass_means = summarry['mean']
hdi_3 = summarry['hdi_3%']
hdi_97 = summarry['hdi_97%']

# Créer un DataFrame à partir des informations extraites
results_df = pd.DataFrame({
    'Espece': biomass_means.index,
    'Biomasse moyenne': biomass_means.values,
    'HDI 3%': hdi_3.values,
    'HDI 97%': hdi_97.values
})

# Mettre 'Identifiant' en tant qu'indice du DataFrame
results_df.set_index('Espece', inplace=True)

# Afficher les résultats avec 'Identifiant' comme indice
results_df

Got error No model on context stack. trying to find log_likelihood in translation.
/home/svivier/.conda/envs/sebpy/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
unknow,1.319,1.014,1.653
Other,4.048,3.813,4.305
0.0001,0.087,0.000,0.197
0.001,0.077,0.000,0.183
0.1,0.140,0.000,0.342
1,1.265,0.000,2.608
Calanus fimarchicus,0.197,0.188,0.207
Calanus glacialis,0.488,0.000,0.963
Calanus hyperboreus,1.692,0.474,2.807


In [64]:
# Initialisation d'une liste pour stocker les données
especes_data = []

# Itérer sur chaque groupe et chaque espèce dans le groupe
for groupe, biomasse_data in results_df.iterrows():
    # Si le groupe est dans le dictionnaire groupes, itérer sur les espèces
    if groupe in groupes:
        for espece in groupes[groupe]:
            especes_data.append({
                'Espece': espece,
                'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
                'HDI 3%': biomasse_data['HDI 3%'],
                'HDI 97%': biomasse_data['HDI 97%']
            })
    else:
        # Si le groupe n'est pas dans groupes, l'ajouter directement comme espèce
        especes_data.append({
            'Espece': groupe,
            'Biomasse moyenne': biomasse_data['Biomasse moyenne'],
            'HDI 3%': biomasse_data['HDI 3%'],
            'HDI 97%': biomasse_data['HDI 97%']
        })

# Créer le DataFrame à partir de la liste de dictionnaires
results_df_especes = pd.DataFrame(especes_data)

# Mettre 'Espèce' en tant qu'indice du DataFrame
results_df_especes.set_index('Espece', inplace=True)

# Afficher les résultats
results_df_especes

,Biomasse moyenne,HDI 3%,HDI 97%
Espece,,,
Euphysa spp.,1.319,1.014,1.653
Spionidae spp.,1.319,1.014,1.653
Limacina retroversa,1.319,1.014,1.653
Limacina spp.,1.319,1.014,1.653
Limacina helicina,1.319,1.014,1.653
...,...,...,...
Microcalanus pusillus,0.028,0.000,0.071
Microcalanus pygmaeus,0.080,0.000,0.193
Pseudocalanus elongatus,0.039,0.028,0.051


In [67]:
groupes = {
    'unknow': ['Euphysa spp.', 'Hyas spp.','Syllides spp.','Cladocera', 'Erythrops erythrophthalma', 'Erythrops spp.', 'Gaidius tenuispinus', 'Hyas', 'Hyperoche medusarum', 'Lepas spp.', 'Staurophora spp.'],
    'Acartia': ['Acartia bifilosa', 'Acartia longiremis', 'Acartia spp.', 'Aglantha digitale', 'Acartia'],
    'Metridia': ['Metridia longa', 'Metridia lucens', 'Metridia spp.'],
    'Microcalanus': ['Microcalanus pusillus', 'Microcalanus pygmaeus', 'Microcalanus spp.'],
    'Temora': ['Temora longicornis', 'Temora spp.'],
    'Oithona': ['Oithona atlantica', 'Oithona similis', 'Oithona spp.', 'Oithona frigida'],
    'Limacina': ['Limacina retroversa', 'Limacina spp.', 'Limacina helicina', 'Clione limacina', 'Limacina','Clione'],
    'Balanus': ['Balanus balanus', 'Balanus crenatus', 'Balanus spp.'],
    'Evadne': ['Evadne nordmanni', 'Evadne spp.'],
    'Sagitta': ['Parasagitta elegans', 'Sagitta spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Centropages': ['Centropages hamatus', 'Centropages spp.', 'Centropages typicus'],
    'Pseudocalanus': ['Pseudocalanus elongatus', 'Pseudocalanus gracilis', 'Pseudocalanus spp.'],
    'Euchaeta': ['Euchaeta spp.'],
    'Eukrohnia': ['Eukrohnia spp.', 'Eukrohnia hamata'],
    'Fritillaria': ['Fritillaria', 'Fritillaria borealis', 'Fritillaria spp.'],
    'Hyperiidea': ['Hyperia spp.', 'Hyperiidea'],
    'Beroe': ['Beroe spp.'],
    'Bolinopsis': ['Bolinopsis spp.'],
    'Bradyidius': ['Bradyidius similis'],
    'Monstrilla': ['Monstrilla spp.'],
    'Oncaea': ['Oncaea borealis', 'Oncaea notopus', 'Oncaea spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Paracalanus': ['Paracalanus spp.'],
    'Paraeuchaeta': ['Paraeuchaeta glacialis', 'Paraeuchaeta norvegica'],
    'Pleurobrachia': ['Pleurobrachia spp.'],
    'Podon': ['Podon spp.', 'Podon leuckartii'],
    'Polychaeta': ['Polychaeta'],
    'Spinocalanus': ['Spinocalanus abyssalis', 'Spinocalanus sp.'],
    'Thalestris': ['Thalestris gibba'],
    'Tisbe': ['Tisbe furcata'],
    'Undinella': ['Undinella oblonga'],
    'Verruca': ['Verruca spp.'],
    'Augaptilus': ['Augaptilus glacialis'],
    'Bradyidius': ['Bradyidius similis'],
    'Chaetognatha': ['Chaetognatha'],
    'Obelia': ['Obelia geniculata',  'Obelia longissima', 'Obelia spp.'],
    'Oikopleura': ['Oikopleura', 'Oikopleura labradoriensis', 'Oikopleura spp.'],
    'Pagurus': ['Pagurus spp.'],
    'Plotocnide': ['Plotocnide borealis', 'Plotocnide spp.'],
    'Scaphocalanus': ['Scaphocalanus brevicornis', 'Scaphocalanus magnus'],
    'Semibalanus': ['Semibalanus balanoides'],
    'Themisto': ['Themisto abyssorum', 'Themisto spp.'],
    'Thysanoessa': ['Thysanoessa inermis', 'Thysanoessa longicaudata', 'Thysanoessa raschii', 'Thysanoessa spp.'],
    'Undinopsis': ['Undinopsis spp.'],
    'Meganyctiphanes': ['Meganyctiphanes norvegica', 'Meganyctiphanes spp.'],
    'Harpacticus': ['Harpacticus littoralis', 'Harpacticus spp.'],
    'Sarsia':['Sarsia princeps', 'Sarsia spp.', 'Sarsia tubulosa'],
    'Spinocalanus': ['Spinocalanus magnus', 'Spinocalanus spinosus','Spinocalanus abyssalis'],
    'Zaus': ['Harpacticus chelifer', 'Zaus goodsiri', 'Zaus spp.'],
    'Parathalestris': ['Parathalestris croni', 'Parathalestris harpactoides'],
    'Rathkea': ['Rathkea octopunctata', 'Rathkea spp.'],
    
    
    
    


    


    
    

    

    
}

# Créer un DataFrame pour stocker les résultats groupés
grouped_results = pd.DataFrame(columns=['Biomasse moyenne (group 5%)', 'HDI 3% (group 5%)', 'HDI 97% (group 5%)'])

# Itérer sur les groupes pour calculer les statistiques
for groupe, especes in groupes.items():
    if any(espece in results_df_especes.index for espece in especes):
        data_subset = results_df_especes.loc[results_df_especes.index.intersection(especes)]
        grouped_results.loc[groupe] = {
            'Biomasse moyenne (group 5%)': data_subset['Biomasse moyenne'].mean(),
            'HDI 3% (group 5%)': data_subset['HDI 3%'].mean(),
            'HDI 97% (group 5%)': data_subset['HDI 97%'].mean()
        }

# Traiter les espèces non groupées
for espece in results_df_especes.index:
    if espece not in sum(groupes.values(), []):
        grouped_results.loc[espece] = {
            'Biomasse moyenne (group 5%)': results_df_especes.at[espece, 'Biomasse moyenne'],
            'HDI 3% (group 5%)': results_df_especes.at[espece, 'HDI 3%'],
            'HDI 97% (group 5%)': results_df_especes.at[espece, 'HDI 97%']
        }

# Afficher les résultats
grouped_results

,Biomasse moyenne (group 5%),HDI 3% (group 5%),HDI 97% (group 5%)
unknow,1.319000,1.014000,1.653000
Acartia,0.080200,0.000000,0.185000
Metridia,0.125000,0.000333,0.272667
Microcalanus,0.061667,0.000000,0.149000
Temora,0.081000,0.000000,0.189000
...,...,...,...
Aetideopsis multiserrata,0.140000,0.000000,0.342000
Calanus spp.,0.140000,0.000000,0.342000
Calanus finmarchicus,0.197000,0.188000,0.207000
Calanus glacialis,0.488000,0.000000,0.963000


In [68]:
# Example: renaming specific columns
grouped_results = grouped_results.rename(columns={
    'Biomasse moyenne (group 5%)': 'mean (6group+)',
    'HDI 3% (group 5%)': 'HDI 3% (6group+)',
    'HDI 97% (group 5%)': 'HDI 97% (6group+)'
    # Add more columns as needed
})


In [70]:
R4= grouped_results.copy()

In [71]:
table = pd.concat([R1, R2, R3, R4], axis=1), results_df = pd.concat([results_df, SpeciesCmass], axis=1)

TypeError: cannot concatenate object of type '<class 'method'>'; only Series and DataFrame objs are valid

In [72]:
R1

,Biomasse moyenne,HDI 3%,HDI 97%,Value
unknow,0.799,0.001,1.862,1.0000
Limacina,0.915,0.386,1.459,1.0000
Undinopsis,0.788,0.000,1.828,1.0000
Balanus,4.754,3.881,5.670,5.0000
Evadne,4.867,3.248,6.535,5.0000
...,...,...,...,...
Tegastes falcatus,0.001,0.000,0.003,0.0010
Mimocalanus distinctocephalus,0.092,0.000,0.205,0.0300
Pseudochirella spectabilis,0.091,0.000,0.213,0.0300
Discoconchoecia elegans,0.080,0.000,0.190,0.0015


In [73]:
R2

<bound method NDFrame.copy of                                Biomasse moyenne  HDI 3%  HDI 97%
Espece                                                          
unknow                                    0.814   0.000    1.911
Limacina                                  0.825   0.269    1.373
Undinopsis                                1.141   0.024    2.386
Balanus                                   3.805   2.991    4.774
Evadne                                    2.568   1.268    3.941
...                                         ...     ...      ...
Tegastes falcatus                         0.794   0.000    1.906
Mimocalanus distinctocephalus             0.792   0.002    1.873
Pseudochirella spectabilis                0.795   0.000    1.821
Discoconchoecia elegans                   0.796   0.001    1.878
Spinocalanus sp.                          0.794   0.000    1.880

[80 rows x 3 columns]>

In [74]:
R3

,Biomasse moyenne (group 5%),HDI 3% (group 5%),HDI 97% (group 5%)
unknow,1.106,0.786,1.430
Acartia,0.040,0.028,0.052
Metridia,0.191,0.181,0.200
Microcalanus,0.040,0.028,0.052
Temora,0.008,0.000,0.022
...,...,...,...
Chiridius obtusifrons,0.191,0.181,0.200
Candacia armata,0.191,0.181,0.200
Aetideopsis multiserrata,0.191,0.181,0.200
Calanus spp.,0.191,0.181,0.200


In [75]:
R4

,mean (6group+),HDI 3% (6group+),HDI 97% (6group+)
unknow,1.319000,1.014000,1.653000
Acartia,0.080200,0.000000,0.185000
Metridia,0.125000,0.000333,0.272667
Microcalanus,0.061667,0.000000,0.149000
Temora,0.081000,0.000000,0.189000
...,...,...,...
Aetideopsis multiserrata,0.140000,0.000000,0.342000
Calanus spp.,0.140000,0.000000,0.342000
Calanus finmarchicus,0.197000,0.188000,0.207000
Calanus glacialis,0.488000,0.000000,0.963000
